In [1]:
from tqdm import tqdm
import os
import re
import codecs
import jieba
import pickle
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.client import IndicesClient
import mysql.connector as sql

In [2]:
es = Elasticsearch([{'host':'localhost','port':9200}])

In [3]:
es_idx = IndicesClient(es)

In [20]:
es_idx.delete(index='_all')

{'acknowledged': True}

In [21]:
a = os.popen('''
curl -X PUT 'localhost:9200/main_idx' -d '
{
  "mappings": {
    "docs": {
      "properties": {
        "pure_text": {
          "type": "text",
          "analyzer": "ik_max_word",
          "search_analyzer": "ik_max_word"
        },
        "processed_text": {
          "type": "text",
          "analyzer": "ik_max_word",
          "search_analyzer": "ik_max_word"
        }
      }
    }
  }
}'
''').read()
for line in a.split('\n'):
    print(line)

{"acknowledged":true,"shards_acknowledged":true,"index":"main_idx"}


In [35]:
# res1 = os.popen('''
# curl -X PUT 'localhost:9200/main_idx/docs/1' -d '
# {
#     "pure_text": "中华人民共和国国歌",
#     "processed_text": "中华 人民 共和国 国歌"
# }'
# ''').read()
# for line in res1.split('\n'):
#     print(line)

{"_index":"main_idx","_type":"docs","_id":"1","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"created":true}


In [39]:
# res = os.popen('''
# curl 'localhost:9200/main_idx/docs/_search'  -d '
# {
#   "query" : { "match" : { "pure_text" : "中华人民共和国" }}
# }'
# ''').read()
# for line in res.split('\n'):
#     print(line)

{"took":1,"timed_out":false,"_shards":{"total":5,"successful":5,"skipped":0,"failed":0},"hits":{"total":1,"max_score":0.25811607,"hits":[{"_index":"main_idx","_type":"docs","_id":"1","_score":0.25811607,"_source":
{
    "pure_text": "中华人民共和国国歌",
    "processed_text": "中华 人民 共和国 国歌"
}}]}}


In [22]:
es_idx.exists(index='main_idx')

True

In [2]:
# all_question = pd.read_csv('./data/description.csv')
Herceptin_question = pd.read_excel('./data/Herceptin.xlsx')
# all_question = pd.read_excel('./data/Herceptin_funding_filtered.xlsx')
# all_question = pd.read_excel('./data/Herceptin_patient_filtered.xlsx')
Kadcyla_question = pd.read_excel('./data/Kadcyla.xlsx')
Perjeta_1_question = pd.read_excel('./data/Perjeta_1.xlsx')
Perjeta_2_question = pd.read_excel('./data/Perjeta_2.xlsx')
Tecentriq_question = pd.read_excel('./data/Tecentriq.xlsx')
Xeloda_question = pd.read_excel('./data/Xeloda.xlsx')
No_product_question = pd.read_excel('./data/No_product_merged.xlsx')

all_question = pd.concat([Herceptin_question, Kadcyla_question, Perjeta_1_question, Perjeta_2_question, Tecentriq_question, Xeloda_question, No_product_question])
print('all_question: ', len(all_question))
all_question.head(5)

all_question.dropna(axis=0, inplace=True, subset=['Description'])
all_question.isna().sum()

print('all_question: ', len(all_question))

jieba.load_userdict('./dict data/chinese_word_dict.txt')
chinese_word_dict = pickle.load(open('./dict data/chinese_word_dict.pickle', 'rb'))
stopwords = [line.strip() for line in codecs.open('./stopwords-zh.txt', 'r', encoding='utf8').readlines()]

def keyword_extract(text, word_dict):
#     text = list(jieba.cut(text, cut_all=False))
    text = list(jieba.cut_for_search(text))
    union_text = list(set(text).intersection(set(word_dict)))
    return " ".join(union_text)

all_question['KeyWords'] = all_question.Description.apply(lambda x: keyword_extract(x, chinese_word_dict))

# 去除所有“[]”内的文本
def remove_special_string(text):
    text = re.sub(r'\[[^]]*\]', '', text)
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'（[^）]*\）', '', text)
    text = re.sub(r'\<[^>]*\>', '', text)
#     text = re.sub(r'[0-9]*', '', text)
#     text = re.sub('患者', '', text)
#     text = re.sub('家属', '', text)
#     text = re.sub('咨询', '', text)
#     text = re.sub('询问', '', text)
    return text
def remove_stopwords(text, stopwords):
    text_cutted = jieba.cut(text, cut_all=True)
    text_removed = []
    for word in text_cutted:
        if word in stopwords or word.strip()=="":
            continue
        text_removed.append(word)
    return "  ".join(text_removed)

all_question['Description'] = all_question.Description.apply(lambda x: remove_special_string(x))
all_question['Processed'] = all_question.Description.apply(lambda x: remove_special_string(x))
all_question['Processed'] = all_question.Processed.apply(lambda x: remove_stopwords(x, stopwords))
all_question.head()

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


all_question:  1970
all_question:  1656


Loading model cost 0.487 seconds.
Prefix dict has been built succesfully.


,Case Number,Description,KeyWords,Processed
0,1420656.0,赫赛汀 配了之后结冰了， 能否继续使用。,使用 赫赛汀,赫赛汀 配 之后 结冰 继续 使用
1,1423956.0,赫赛汀粉末形状如何？患者保存的赫赛汀的稀释液出现结冰情况，是否能否使用,无 使用 患者 赫赛汀,赫赛汀 粉末 形状 保存 赫赛汀 稀释 稀释液 出现 结冰 冰情 情况 ...
2,1434184.0,请帮忙查一些赫赛汀与血栓栓塞方面的数据和文献,栓塞 查 赫赛汀 文献 血栓栓塞 无 血栓,帮忙 查 赫赛汀 血栓 血栓栓塞 栓塞 方面 数据 和文 文献
3,1436011.0,拿到的赫赛汀呈现块状是否正常,正常 赫赛汀,拿到 赫赛汀 呈现 块状 是否 正常
4,1438071.0,常规配置和保存的赫赛汀复溶后的溶液保存在2-8度的情况下，现出现浑浊情况，能否使用,赫赛汀 配置 现 下 溶液 使用,常规 配置 保存 赫赛汀 复 溶 溶液 保存 存在 2 8 度 情况...


In [259]:
all_query_text = []
for idx, row in all_question.iterrows():
    all_query_text.append(str(row['Description']) + "\n\n")
    
# all_query_text_f = open('all_query_text.txt', 'w')
# all_query_text_f.writelines(all_query_text)
# all_query_text_f.close()

In [24]:
doc_list = []
for idx,row in all_question.iterrows():
    doc = {"pure_text": row['Description'], "processed_text": row['KeyWords']}
    doc_list.append(doc)
print(len(doc_list))
print(doc_list[0])

1656
{'pure_text': '赫赛汀  配了之后结冰了， 能否继续使用。', 'processed_text': '赫赛汀 使用'}


In [9]:
res1 = os.popen('''
curl -X POST 'localhost:9200/main_idx/docs' -d '
{
    "pure_text": "这是一条样例数据，包含乳腺癌激酶这个词。"
}'
''').read()
for line in res1.split('\n'):
    print(line)

{"_index":"main_idx","_type":"docs","_id":"AWu2fenK8gCSk-fY15qv","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"created":true}


In [27]:
for doc in tqdm(doc_list):
    temp = os.popen('''
    curl -X POST 'localhost:9200/main_idx/docs' -d '
    {}
    '
    '''.format(str(doc).replace("\"", "\'").replace("\'", "\""))).read()
#     for line in temp.split('\n'):
#         print(line)
        
#     print('''
#     curl -X PUT 'localhost:9200/ik_test/docs/{}' -d '
#     {}
#     '
#     '''.format(str(idx), str(doc_list[idx]).replace("\"", "\'").replace("\'", "\"")))
    
# curl -X PUT 'localhost:9200/ik_test/docs/1' -d '
# {
#     "content": "中华人民共和国国歌"
# }'

100%|██████████| 1656/1656 [01:54<00:00, 14.05it/s]


In [ ]:
res = os.popen('''
curl 'localhost:9200/main_idx/docs/_search?pretty=true'  -d '
{
  "query" : { "match" : { "processed_text" : "关于赫赛汀心脏毒性：1、临床 2、临床前" }}
}'
''').read()
for line in res.split('\n'):
    print(line)

In [96]:
for idx in range(len(doc_list)):
    es.index(index='text_search', id=idx, body=doc_list[idx])

In [6]:
query = "乳腺癌激酶"

In [10]:
# processed_text  pure_text
results = es.search(index='main_idx', body=
{
  "query" : { "match" : { "pure_text" : query }}
}
# {
#   "query": {
#     "bool": {
#       "must":     { "match": { "processed_text": query }},
#       "should":   { "match": { "pure_text": "帕妥珠单抗" }}
#     }
#   }
# }
,size=10)
hits = results['hits']['hits']
# results
for idx, res in enumerate(hits):
#     if "帕妥珠单抗" in res['_source']['pure_text']:
    print(str(idx+1) + ":  " + res['_source']['pure_text'])

1:  这是一条样例数据，包含乳腺癌激酶这个词。


In [11]:
tokens_res = es_idx.analyze(index='main_idx', body={
#     "analyzer": "ik_max_word",
    "analyzer": "ik_smart",
    "text" : "这是一条样例数据，包含乳腺癌激酶这个词"
})
for token in tokens_res['tokens']:
    print(token['token'])

这是
一条
样
例
数据
包含
乳腺癌激酶
这个词
